In [2]:
import numpy as np
import pandas as pd
from scipy.linalg import eigh, eig
from scipy.optimize import minimize, differential_evolution
import matplotlib.pyplot as plt
import time

In [6]:
#Rough draft 29 -- chatGPT test 2

bounds = [(-100, 100)] * len(initial_guess)

result = differential_evolution(opt_prelim, bounds, strategy='best1bin', maxiter=1000, popsize=15, tol=1e-7, mutation=(0.5, 1), recombination=0.7)

optimal_params = result.x
optimal_D, optimal_S = modified_DS(optimal_params, params_shape[0], params_shape[1])

print("\nOptimization using Differential Evolution:")
print("Optimal D:\n", optimal_D)
print("Optimal S:\n", optimal_S)

print("\nOriginal D:\n", D)
print("Original S:\n", S)


Optimization using Differential Evolution:
Optimal D:
 [[-63.83785979   0.           0.           0.        ]
 [  0.          95.67585486   0.           0.        ]
 [  0.           0.          76.01510698   0.        ]
 [  0.           0.           0.         -43.69099028]]
Optimal S:
 [[ -3.93889549  -0.7524846    1.98076813 -15.06353977]
 [ -0.7524846    7.28627625  -4.92913     -3.24002845]
 [  1.98076813  -4.92913      2.37563031  10.88858916]
 [-15.06353977  -3.24002845  10.88858916  -2.54818487]]

Original D:
 [[43.7086107   0.          0.          0.        ]
 [ 0.         95.56428758  0.          0.        ]
 [ 0.          0.         75.87945476  0.        ]
 [ 0.          0.          0.         63.87926358]]
Original S:
 [[2.40416776 4.40699289 5.00736814 6.26688275]
 [4.40699289 7.3726532  2.04815622 7.72599828]
 [5.00736814 2.04815622 2.6364247  3.76907288]
 [6.26688275 7.72599828 3.76907288 3.62106226]]


In [5]:
#Rough draft 28 -- addition of differential evolution

# gloabl optimization w/ differential evolution
print("\nOptimization using Differential Evolution:")

opt_result = differential_evolution(opt_prelim, bounds=[(-100, 100)] * len(initial_guess), disp=True)
optimal_params = opt_result.x

optimal_D, optimal_S = modified_DS(optimal_params, params_shape[0], params_shape[1])
opt_results['Differential Evolution'] = (optimal_D, optimal_S)

print("Optimal D (Differential Evolution):\n", optimal_D)
print("Optimal S (Differential Evolution):\n", optimal_S)


Optimization using Differential Evolution:
differential_evolution step 1: f(x)= 150661
differential_evolution step 2: f(x)= 150661
differential_evolution step 3: f(x)= 150661
differential_evolution step 4: f(x)= 150661
differential_evolution step 5: f(x)= 150661
differential_evolution step 6: f(x)= 98810.4
differential_evolution step 7: f(x)= 96672.5
differential_evolution step 8: f(x)= 91707
differential_evolution step 9: f(x)= 91707
differential_evolution step 10: f(x)= 91707
differential_evolution step 11: f(x)= 91707
differential_evolution step 12: f(x)= 88161
differential_evolution step 13: f(x)= 69237.8
differential_evolution step 14: f(x)= 69237.8
differential_evolution step 15: f(x)= 69237.8
differential_evolution step 16: f(x)= 69237.8
differential_evolution step 17: f(x)= 69237.8
differential_evolution step 18: f(x)= 56577.5
differential_evolution step 19: f(x)= 56577.5
differential_evolution step 20: f(x)= 42987
differential_evolution step 21: f(x)= 42987
differential_evolu

In [5]:
#Rough draft 27 orig -- new rough draft, fixing local minima error

#PART1

# nec_var1
np.random.seed(42)
n = 2
alpha_values = np.linspace(-1, 1, 100)

# nec_mat for M_true(a) = D + a * S
def nec_mat(n):
    D = np.diag(np.random.uniform(1, 10, n)) #####
    A = np.random.uniform(1, 10, (n, n))
    S = np.abs(A + A.T) / 2
    return D, S
D, S = nec_mat(n)

# nec_mat (N and v0) for generalized_eigen
def N_prelim(n):
    half_n = n // 2
    return np.diag([1] * half_n + [-1] * half_n)
N = N_prelim(n)
v0 = np.ones(n)

# generalized_eigen for M_true(a)
def generalized_eigen(D, S, N, alpha):
    M_true = D + alpha * S
    eigenvalues, eigenvectors = eig(M_true, N)
    return np.real(eigenvalues), np.real(eigenvectors)

# data_table_prelim (alpha, eigenvalues, B, eigenvectors)
def data_table_prelim(D, S, N, alpha_values, v0):
    data = []
    
    for alpha in alpha_values:
        eigenvalues, eigenvectors = generalized_eigen(D, S, N, alpha)
        
        eigenvalues = np.sort(np.real(eigenvalues))        
        B = [np.dot(eigenvectors[:, i], v0) for i in range(eigenvectors.shape[1])]
        
        row = list(eigenvalues) + list(B) + eigenvectors.flatten().tolist()
        data.append(row)
        
    eigenvalue_columns = [f"E{i + 1}" for i in range(D.shape[0])]
    B_columns = [f"B{i + 1}" for i in range(D.shape[0])]
    eigenvector_columns = [f"V{i + 1}_{j + 1}" for i in range(D.shape[0]) for j in range(D.shape[0])]
    columns = eigenvalue_columns + B_columns + eigenvector_columns
    
    df = pd.DataFrame(data, columns=columns)
    return df

data_table = data_table_prelim(D, S, N, alpha_values, v0)
#print("Data Table:\n", data_table)

#PART2

# nec_var2
P_values = np.linspace(-20, 20, 500)

#modified_DS
def modified_DS(params, D_shape, S_shape):
    D_mod = np.diag(params[:D_shape[0]])
    S_mod = np.zeros(S_shape)
    
    upper_tri_indices = np.triu_indices(S_shape[0])
    S_mod[upper_tri_indices] = params[D_shape[0]:D_shape[0] + len(upper_tri_indices[0])]
    
    S_mod += np.tril(S_mod.T, -1)
    return D_mod, S_mod

#cost_function
def cost_function(params, D_shape, S_shape, N, alpha_values, data_table):
    D_mod, S_mod = modified_DS(params, D_shape, S_shape)
    
    total_cost = 0
    
    for idx, alpha in enumerate(alpha_values):
        eigenvalues, eigenvectors = generalized_eigen(D_mod, S_mod, N, alpha)
        
        eigenvalues = np.sort(np.real(eigenvalues))
        true_eigenvalues = data_table.iloc[idx, :n].values
        B = [np.dot(eigenvectors[:, i], v0) for i in range(eigenvectors.shape[1])]
        
        true_B = data_table.iloc[idx, n:2*n].values
        true_eigenvectors = data_table.iloc[idx, 2*n:].values.reshape(n, n)
        
        total_cost += np.sum((eigenvalues - true_eigenvalues) ** 2) \
        + np.sum((B - true_B) ** 2)
        total_cost += np.sum((eigenvectors.flatten() - true_eigenvectors.flatten()) ** 2)
        
    return total_cost

#PART3

#optimization
#initial_guess = np.concatenate([np.diag(D), S[np.triu_indices(n)]]) #***

nec_num_param = n + int(n * (n + 1) / 2)
random_initial_guess = np.random.uniform(1, 10, nec_num_param)
print("random_initial_guess:", random_initial_guess)
params_shape = (D.shape, S.shape)

# opt_prelim for calibration of D and S
def opt_prelim(params):
    #return cost_function(params, *params_shape, N, alpha_values, data_table)
    return cost_function(params, params_shape[0], params_shape[1], N, alpha_values, data_table)

#---------------- method editions added below

methods = ['L-BFGS-B', 'Nelder-Mead', 'Powell']

opt_results = {}

for method in methods:
    print(f"\nOptimization using {method}:")
    opt_result = minimize(opt_prelim, random_initial_guess, method=method, options={'disp': True}) #####
    optimal_params = opt_result.x
    #optimal_D, optimal_S = modified_DS(optimal_params, *params_shape)
    optimal_D, optimal_S = modified_DS(optimal_params, params_shape[0], params_shape[1])
    opt_results[method] = (optimal_D, optimal_S)
    
    print("Optimal D:\n", optimal_D)
    print("Optimal S:\n", optimal_S)

print("\nOriginal D:\n", D)
print("Original S:\n", S)

cost_values = [opt_prelim(np.concatenate([[P], random_initial_guess[1:]])) for P in P_values]

random_initial_guess: [1.52275251 8.79558531 6.41003511 7.3726532  1.18526045]

Optimization using L-BFGS-B:
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.14498D+03    |proj g|=  8.04384D+02

At iterate    1    f=  1.21890D+03    |proj g|=  5.89091D+02

At iterate    2    f=  1.36683D+02    |proj g|=  1.11104D+02


 This problem is unconstrained.



At iterate    3    f=  5.78617D+01    |proj g|=  2.81406D+01

At iterate    4    f=  5.05262D+01    |proj g|=  1.76837D+01

At iterate    5    f=  3.90859D+01    |proj g|=  2.24447D+01

At iterate    6    f=  1.80719D+01    |proj g|=  1.86670D+01

At iterate    7    f=  1.51192D+00    |proj g|=  1.89395D+01

At iterate    8    f=  7.62506D-01    |proj g|=  1.01706D+01

At iterate    9    f=  4.83788D-02    |proj g|=  2.83580D+00

At iterate   10    f=  1.59767D-03    |proj g|=  4.48498D-01

At iterate   11    f=  8.75341D-05    |proj g|=  8.33343D-02

At iterate   12    f=  2.99723D-06    |proj g|=  1.21734D-02

At iterate   13    f=  1.39460D-08    |proj g|=  1.83747D-03

At iterate   14    f=  9.01217D-10    |proj g|=  4.94362D-04
Optimal D:
 [[4.37086105 0.        ]
 [0.         9.55642872]]
Optimal S:
 [[7.58794618 4.39604688]
 [4.39604688 2.40395008]]

Optimization using Nelder-Mead:

At iterate   15    f=  2.96855D-11    |proj g|=  4.70189D-05

           * * *

Tit   = total nu

In [14]:
n + int(n * (n + 1) / 2)

In [16]:
orig_eigenvalues = data_table.iloc[alpha_values[0], :n].values
orig_dot_products = data_table.iloc[alpha_values[0], n:2*n].values

opt_D, opt_S = opt_results['L-BFGS-B']



opt_eigenvalues, opt_eigenvectors = generalized_eigen(opt_D, opt_S, N, alpha)
opt_dot_products = [np.dot(opt_eigenvectors[:, i], v0) for i in range(opt_eigenvectors.shape[1])]




fig, ax = plt.subplots()
ax.plot(orig_eigenvalues, orig_dot_products, 'bo-', label='Original', markersize=10, linewidth=2)
ax.set_xlabel('Eigenvalue/Pole')
#ax.set_ylabel('Dot Product/Height')
ax.set_title(f'Eigenvalues and Dot Products = {alpha}')
ax.legend()
plt.show()

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [17]:
print(alpha)

-1.0
